In [1]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from numpy import array
from collections import Counter
from scipy.sparse import csr_matrix

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary

import re

In [2]:
# Use spacy lib
# On https://spacy.io/

import spacy
nlp = spacy.load('fr')

In [3]:
##############
# Parameters #
##############

min_gram = 1
max_gram = 3

# To create ours partitions, we must first know the years which will be the limits
limit_years = [2007, 2010, 2014]

# Ignore words that appear at a frequency less than tresh_freq in the corpus
tresh_freq = 0.8

# Number of clusters by partitions
nb_clusters = 5

In [4]:
# Datas preprocessing methods.

# Lemmatisation without poncutations

stemmer = nltk.stem.snowball.FrenchStemmer()
fstw = stopwords.words('french')

# French Stop Words, extraits depuis le fichier stopwords-fr.txt + stopwords french de nltk
sourceFST = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()]+fstw

# Based on ration of french and english stopwords
def isEnglish(article):
    total_fsw = len([x for x in article.split() if x in sourceFST])
    total_esw = len([x for x in article.split() if x in stopwords.words('english')])
    ratio = 100
    if total_fsw != 0:
        ratio = total_esw/total_fsw
    return ratio > 1 and total_esw > 3

def lemmatize(article):
    artiregex = re.sub(" [0-z][0-z] ", " ", article) # word of length < 2
    artiregex = artiregex.lower()
    artiregex = re.sub("(é|è|ê)", "e", artiregex)
    output = []
    outPonc = artiregex.translate(artiregex.maketrans("","", string.punctuation))
    outLem = nlp(outPonc)
    for token in outLem:
        if token.lemma_ not in sourceFST and [x for x in token.lemma_ if x not in "0123456789"] != []:
            output.append(token.lemma_)
    res = ' '.join(output)
    return res

In [5]:
# Data Reading
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

In [6]:
# Let's process our corpus, and determine a limit to split it in partitions

# usable[] correspond to our corpus processed
# limits[] let us know when to delimit partitions
limits = []
usable = []

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(data['abstract']), 1):
    #if not null, empty, or whatever (so if there is a abstract):
    if not isinstance(data['abstract'][i], float) and not isEnglish(data['abstract'][i]):
        text = data['abstract'][i]
        if not isinstance(data['title'][i], float):
            text += " "+data['title'][i]

        numArti+=1
        usable.append(stemmer.stem(lemmatize(text)))
        year = data['year'][i]
        if year != prev_year:
            prev_year = year
            if year in limit_years:
                limits.append(numArti-1)
limits.append(numArti-1)


In [7]:
# Display pre-processed datas

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram), max_df=tresh_freq)
tfidf = vectorizer.fit_transform(usable)

print("nombre d'articles =", len(usable))
print("nombre de mots =", len(tfidf.toarray()[0]))
print("limits =", limits)

usable[0]

/info/etu/m2/i140302/venv/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['quelqu'] not in stop_words.
  sorted(inconsistent))


nombre d'articles = 991
nombre de mots = 133044
limits = [222, 467, 693, 990]


'plateforme objectif permettre citoyen danalyserpar euxmemer tweet politique devenement specifiqu francepour cas lelection presidentiell ideo2017 analyser quasitemps reel message candidat fournir principal caracteristiqueslusage lexiqu politique comparaison entrer candidat ideo2017   plateforme citoyen dediee lanalyse tweet evenement polit'

In [8]:
# Creation of partitions_tfidf[], which give us the TFIDF of each cluster of each partition
# partitions_tfidf[num_partition][num_doc][num_word]
# Beware, num_doc can't be equals to 1091 (max). You have partitions, so every doc aren't in every partitions
# num_word can be found via vectorizer.get_feature_name()
partitions_tfidf = []
beg = 0
for l in limits:
    last = l+1
    partitions_tfidf.append([list(x) for x in list(tfidf.toarray())[beg:last]])
    beg = l

In [9]:
len(partitions_tfidf)

4

# KMeans

In [10]:
# Applying KMeans on tfidf
# the labels_ give assignment of doc to the cluster number 
km = KMeans(n_clusters=nb_clusters)
km.fit(tfidf)
cluster = km.labels_

cluster_partition = [cluster[:limits[0]],cluster[limits[0]:limits[1]],cluster[limits[1]:limits[2]],cluster[limits[2]:limits[3]]]

In [11]:
part_km = []
for i in range(0, len(limits)):
    dash = km.fit(partitions_tfidf[i])
    part_km.append(dash)

In [12]:
#(part_km[1]).labels_
#
#all_labels = []
#for i in range(0,len(limits)):
#    for j in range(0, len((part_km[i]).labels_)):
        

In [13]:
# doc_clustering is a dictionnary 
# it looks like -> { doc_number : [partition_number, cluster_number] }
# This is used to reassign doc number to their respective partition and and cluster
doc_clustering = {}
for i in range(0,len(usable)):
    if i < limits[0]:
        doc_clustering[i] = [0, cluster[i]]
    elif i >= limits[0] and i < limits[1]:
        doc_clustering[i] = [1, cluster[i]]
    elif i >= limits[1] and i < limits[2]:
        doc_clustering[i] = [2, cluster[i]]
    else:
        doc_clustering[i] = [3, cluster[i]]

In [14]:
# Allows to get list of documents number
# return [dou numbers]
# params : partition_number , cluster number
partitions = []
def get_doc(part, clust):
    docs = []
    for i in range(0,len(doc_clustering)):
        if doc_clustering[i][0] == part and doc_clustering[i][1] == clust:
            docs.append(i)
    return docs

In [15]:
# Get the partitions variable
# Here partitions[part][cluster] = list of docs numbe
partitions = []
for i in range(0, len(limits)):
    clusters = []
    for j in range(0, nb_clusters):
        clusters.append(get_doc(i,j))
    partitions.append(clusters)

In [16]:
# example of output for doc_clustering
# doc 465 is in cluster 1 of the partition 1
# doc 154 is in cluster 2 of the partition 0

print(doc_clustering[465])
print(doc_clustering[154])
print()

# Here, just count docs number by cluster.
print(Counter(cluster_partition[0]))
print(Counter(cluster_partition[1]))
print(Counter(cluster_partition[2]))
print(Counter(cluster_partition[3]))

[1, 3]
[0, 3]

Counter({3: 66, 1: 63, 4: 39, 0: 33, 2: 21})
Counter({3: 71, 1: 63, 4: 54, 0: 31, 2: 26})
Counter({3: 75, 1: 50, 4: 49, 0: 28, 2: 24})
Counter({3: 87, 1: 62, 4: 61, 2: 56, 0: 31})


# Quality Measure

In [17]:
# INSERT QUALITY MEASURE HERE

# Khi²

In [18]:
# tf_of_your_word = tf[numDoc][strWord]
tf = []
for doc in usable:
    tf_doc = {}
    for word in vectorizer.get_feature_names():
        tf_doc[word] = doc.count(word)
    tf.append(tf_doc)

In [19]:
# Number total of words
# nb_total_word[numPartition]
nb_total_word = []
nb = 0

for numDoc in range(0, len(usable)):
    for word in vectorizer.get_feature_names():
        nb += tf[numDoc][word]
    if numDoc in limits:
        nb_total_word.append(nb)
        nb=0
    

In [20]:
nb_total_word

[104156, 111095, 98925, 131037]

In [21]:
tf[0]

{'definis detablir correspondance': 0,
 'permettre lanalys brevet': 0,
 'criteria using the': 0,
 'deffectuer pretraitemer': 0,
 'graduel sou contraint': 0,
 'lintegration connaissancer sou': 0,
 'amenagement effectuer': 0,
 'cluster garantir': 0,
 'redondancer dan tableau': 0,
 'cancer nasopharynxnpc lobjectif': 0,
 'synthese': 0,
 'genericite system': 0,
 'matrice apparente matrice': 0,
 'supervisee lalgorithm': 0,
 'besoin outil specialis': 0,
 'chaïne': 0,
 'cohesion degr deligibilite': 0,
 'difficulteeter': 0,
 'initial requete equivalent': 0,
 'automatiquede schema': 0,
 'enrestreigner': 0,
 'aborder mentionnerqu': 0,
 'production issu': 0,
 'contexte dextraction formel': 0,
 'librement accessible web': 0,
 'gestion donnee prive': 0,
 'structurel poindre depart': 0,
 'consistance operateur': 0,
 'echangeur': 0,
 'enjeu importer': 0,
 'modifier lalgorithme kohonen': 0,
 'appliqu donnee cellesci': 0,
 'apport regle': 0,
 'lavantage detr insensible': 0,
 'granularite tr': 0,
 'despa

In [22]:
# nb_word[num_partition][word]
nb_word = []

word_in_this_parti = {}
for word in vectorizer.get_feature_names():
    word_in_this_parti[word] = 0

for numDoc in range(0, len(usable)):
    for word in vectorizer.get_feature_names():
        word_in_this_parti[word] += tf[numDoc][word]
    if numDoc in limits:
        nb_word.append(word_in_this_parti)
        word_in_this_parti = {}
        for word in vectorizer.get_feature_names():
            word_in_this_parti[word] = 0

In [23]:
len(nb_word)

4

In [25]:
# nb_word_by_cluster[numPartition][numCluster]
nb_word_by_cluster = []
for parti in partitions:
    nb_word_clus = []
    for cluster in parti:
        nb = 0
        for numDoc in cluster:
            for word in vectorizer.get_feature_names():
                nb += tf[numDoc][word]
        nb_word_clus.append(nb)
    nb_word_by_cluster.append(nb_word_clus)

In [28]:
# Expected values, if nothing were dependant
# exp[numPartition][numCluster][numWord]
#exp = []
#for numParti in range(0, len(partitions)):
#    exp_clus = []
#    for numCluster in range(0, len(partitions[numParti])):
#        exp_word = []
#        for numWord in range(0, vectorizer.get_feature_names()):
#            exp_word.append((nb_word[numParti][numWord] + nb_word_by_cluster[numPart][numCluster]) / nb_total_word[numParti])
#        exp_cluster.append(exp_word)
#    exp.append(exp_clus)


In [30]:
# value_of_khi2 = khi2[numPartition][numCluster][word]
khi2 = []

for numParti in range(0, len(partitions)):
    khi2parti = []
    for numCluster in range(0, len(partitions[numParti])):
        khi2cluster = {}
        
        for word in vectorizer.get_feature_names():
            word_in_this_parti[word] = 0
            E = (nb_word[numParti][word] + nb_word_by_cluster[numParti][numCluster])/nb_total_word[numParti]
            N = 0
            for numDoc in partitions[numParti][numCluster]:
                N += tf[numDoc][word]
            khi2cluster[word] = (pow(N - E, 2)/E)        
        khi2parti.append(khi2cluster)
    khi2.append(khi2parti)

In [33]:
# list of your labels = labels[numPartition][numCluster]
labels = []

for numPartition in range(0, len(nb_word_by_cluster)):
    label_clus = []
    for numCluster in range(0, len(nb_word_by_cluster[numPartition])):
        label_clus.append(Counter(khi2[numPartition][numCluster]).most_common(5))
    labels.append(label_clus)

In [34]:
labels

[[[('io', 420079.77910686866),
   ('an', 369541.7093965188),
   ('is', 252125.3683190454),
   ('tr', 211494.70491828723),
   ('pr', 193885.75209968883)],
  [('io', 872009.7007252688),
   ('an', 681359.4506458181),
   ('pr', 558811.5603648517),
   ('is', 499992.7237225606),
   ('tr', 451485.20113461884)],
  [('io', 364633.04654272436),
   ('is', 282791.1478496944),
   ('pr', 158166.95058697282),
   ('an', 155752.02209170282),
   ('li', 145903.4206025257)],
  [('io', 989605.6665852761),
   ('an', 709452.9757324123),
   ('pr', 587937.7323024924),
   ('is', 579378.917415738),
   ('li', 508954.1969741451)],
  [('io', 476500.66439288703),
   ('an', 329721.1178821748),
   ('tr', 294965.1893938269),
   ('li', 277937.7049822384),
   ('pr', 266174.8631516363)]],
 [[('io', 438144.0294241331),
   ('an', 231100.75711450452),
   ('is', 225989.1328186693),
   ('pr', 196265.92470999752),
   ('tr', 184702.2412414458)],
  [('io', 1220779.3217363046),
   ('is', 718420.4254007118),
   ('an', 637486.937731